In [2]:
import numpy as np
import pandas as pd
import seaborn
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams.update({'font.size':20})
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "sans"



In [2]:
#read the train data
train_data = pd.read_csv('house_predictions_train.csv', index_col = 0) 
test_data =pd.read_csv('house_predictions_test.csv', index_col = 0)  #index_col meaning: remove the first column

In [3]:
train_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
train_data.info() #the data is made of three 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-

# Cleaning Data

In [5]:
#train_data.columns

In [6]:
#find out numeric  columns

numeric=['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
train_num = train_data.select_dtypes(include=numeric)


In [7]:
#find out categorical   columns

train_cat=train_data.select_dtypes(include='object')


Find out the distribution of sales price and skewness in the data

The sales price is distributed around the number 200000 
(one way would be to group the number of houses to see how many houses has prices less than this and higher than this)

The sales price is skewed and is not normally distributed this should be taken into occount while applying the linear regression models on it

# Data engineering 

Combine train and tes tdata set to aovid apllying methods 2 times

In [8]:

ntrain = train_data.shape[0]
ntest = test_data.shape[0]

all_data = pd.concat((train_data, test_data), sort = True).reset_index(drop=True)

nindex, nfeatures = all_data.shape

In [9]:
all_data.tail()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,...,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
2914,546,546,0,NaN,3,Twnhs,TA,No,0.0,0.0,...,WD,0,Pave,5,546.0,AllPub,0,1970,1970,2006
2915,546,546,0,NaN,3,TwnhsE,TA,No,252.0,0.0,...,WD,0,Pave,6,546.0,AllPub,0,1970,1970,2006
2916,1224,0,0,NaN,4,1Fam,TA,No,1224.0,0.0,...,WD,0,Pave,7,1224.0,AllPub,474,1960,1996,2006
2917,970,0,0,NaN,3,1Fam,TA,Av,337.0,0.0,...,WD,0,Pave,6,912.0,AllPub,80,1992,1992,2006
2918,996,1004,0,NaN,3,1Fam,TA,Av,758.0,0.0,...,WD,0,Pave,9,996.0,AllPub,190,1993,1994,2006


Based on the above graph, the features that has more than 50% missing values are alley, Fence , FireplaceQu, MiscFeature, PoolQC,salesPrice

In [10]:
# Checking Missing Values Column 
all_data[["LotFrontage", "Alley", "MasVnrType", "MasVnrArea", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "Electrical", "FireplaceQu", "GarageType", "GarageYrBlt", "GarageFinish", "GarageQual", "GarageCond", "PoolQC", "Fence", "MiscFeature"]].head(10)

,LotFrontage,Alley,MasVnrType,MasVnrArea,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MiscFeature
0,65.0,NaN,BrkFace,196.0,Gd,TA,No,GLQ,Unf,SBrkr,NaN,Attchd,2003.0,RFn,TA,TA,NaN,NaN,NaN
1,80.0,NaN,None,0.0,Gd,TA,Gd,ALQ,Unf,SBrkr,TA,Attchd,1976.0,RFn,TA,TA,NaN,NaN,NaN
2,68.0,NaN,BrkFace,162.0,Gd,TA,Mn,GLQ,Unf,SBrkr,TA,Attchd,2001.0,RFn,TA,TA,NaN,NaN,NaN
3,60.0,NaN,None,0.0,TA,Gd,No,ALQ,Unf,SBrkr,Gd,Detchd,1998.0,Unf,TA,TA,NaN,NaN,NaN
4,84.0,NaN,BrkFace,350.0,Gd,TA,Av,GLQ,Unf,SBrkr,TA,Attchd,2000.0,RFn,TA,TA,NaN,NaN,NaN
5,85.0,NaN,None,0.0,Gd,TA,No,GLQ,Unf,SBrkr,NaN,Attchd,1993.0,Unf,TA,TA,NaN,MnPrv,Shed
6,75.0,NaN,Stone,186.0,Ex,TA,Av,GLQ,Unf,SBrkr,Gd,Attchd,2004.0,RFn,TA,TA,NaN,NaN,NaN
7,NaN,NaN,Stone,240.0,Gd,TA,Mn,ALQ,BLQ,SBrkr,TA,Attchd,1973.0,RFn,TA,TA,NaN,NaN,Shed
8,51.0,NaN,None,0.0,TA,TA,No,Unf,Unf,FuseF,TA,Detchd,1931.0,Unf,Fa,TA,NaN,NaN,NaN
9,50.0,NaN,None,0.0,TA,TA,No,GLQ,Unf,SBrkr,TA,Attchd,1939.0,RFn,Gd,TA,NaN,NaN,NaN


# Replacing the columns with missing values individually

In [11]:
#Handling the Nan values for each column individually

In [12]:
#there is no order in the values so replace nan with Nane
all_data.Alley.unique()  
all_data['Alley'] = all_data['Alley'].fillna('None')



Grvl  =   Gravel
Pave  =   Paved
NA    =   No alley access



In [13]:
all_data.Fence.unique()  


array([nan, 'MnPrv', 'GdWo', 'GdPrv', 'MnWw'], dtype=object)

In [14]:
all_data['Fence'] = all_data['Fence'].fillna('None')

		
- GdPrv	Good Privacy
- MnPrv	Minimum Privacy
- GdWo	Good Wood
- MnWw	Minimum Wood/Wire
- NA	No Fence

In [15]:
all_data.FireplaceQu.unique()  

array([nan, 'TA', 'Gd', 'Fa', 'Ex', 'Po'], dtype=object)

- Ex	Excellent - Exceptional Masonry Fireplace
- Gd	Good - Masonry Fireplace in main level
- TA	Average - Prefabricated Fireplace in main living area or Masonry Fireplace in basement
- Fa	Fair - Prefabricated Fireplace in basement
- Po	Poor - Ben Franklin Stove
- NA	No Fireplace

In [16]:
all_data['FireplaceQu'] = all_data['FireplaceQu'].fillna('None')


In [17]:

quality_map = {
    "nan": 0, "Po": 1, "FA": 2, "TA": 3, "Gd": 4, "Ex": 5
}

all_data['FireplaceQu'].replace(quality_map, inplace=True)

In [18]:
all_data.MiscFeature.unique()

array([nan, 'Shed', 'Gar2', 'Othr', 'TenC'], dtype=object)

In [19]:
all_data['MiscFeature'] = all_data['MiscFeature'].fillna('None')

- Elev	Elevator
- Gar2	2nd Garage (if not described in garage section)
 - Othr	Other
- Shed	Shed (over 100 SF)
- TenC	Tennis Court
- NA	None
		

In [20]:
all_data.PoolQC.unique()

array([nan, 'Ex', 'Fa', 'Gd'], dtype=object)

- Ex	Excellent
- Gd	Good
- TA	Average/Typical
- Fa	Fair
- NA	No Pool

In [21]:
all_data['PoolQC'] = all_data['PoolQC'].fillna('None')

quality_map = {
    "None": 0, "Fa": 1, "Gd": 2, "Ex": 3
}

all_data['PoolQC'].replace(quality_map, inplace=True)

In [22]:
all_data['LotFrontage'].dtype

dtype('float64')

In [23]:
all_data['LotFrontage'].fillna(value=all_data['LotFrontage'].mean(), inplace=True)
#replace missing values with their mean values

In [24]:
df_no_garage = all_data[[
    'GarageYrBlt', 'GarageCond', 'GarageFinish', 'GarageQual', 'GarageType', 'GarageCars', 'GarageArea'
]][
    all_data.isnull()
]

df_no_garage.head(5)


,GarageYrBlt,GarageCond,GarageFinish,GarageQual,GarageType,GarageCars,GarageArea
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In the above dataframe we can notice that when the feature GarageTpe is NaN, the corresponding categorical Garage features are also NaN and the numerical features are 0. This clearly indicates absence of garage for the house. and the total number entries with Null garage Type is 157...

now we can safely impute the NaN values with string value of None for all the above features.


In [25]:
print(f"Total number of entries with Null Garage Type are {len(df_no_garage.index)}.")

Total number of entries with Null Garage Type are 2919.


In [26]:
for feature in ['GarageCond', 'GarageFinish', 'GarageQual', 'GarageType']:
    all_data[feature].fillna(value='None', inplace=True)
    
for feature in ['GarageYrBlt', 'GarageCars', 'GarageArea']:
    all_data[feature].fillna(value=0, inplace=True)

#NOTE: The additional two features GarageCars and GarageArea have a value of 0 for the corresponding NaN values.

In [27]:
no_basement_df = all_data[[
    'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtQual', 'BsmtFinSF1', 
    'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF', 'TotalBsmtSF'
]][
    all_data.isnull()
]

no_basement_df.head(5)

,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,BsmtQual,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,TotalBsmtSF
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
#imputing NaN value with string value of None for all the categorical features and 0 for numeric variables.
# Handling all Null in Basement variable
for feature in ['BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtQual']:
    all_data[feature].fillna(value='None', inplace=True)
    
for feature in ['BsmtFinSF1', 'BsmtFinSF2','BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF', 'TotalBsmtSF']:
    all_data[feature].fillna(value=0, inplace=True)

In [29]:
all_data['MasVnrType'].unique()

array(['BrkFace', 'None', 'Stone', 'BrkCmn', nan], dtype=object)

In [30]:
#replace nan with None
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
#replace nan with zero
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)

In [31]:
#all_data["MasVnrArea"].unique()

In [32]:
all_data['MSZoning'].isnull().sum() #for missing values in this column

4

In [33]:
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])

In [34]:
all_data.Utilities.unique()

array(['AllPub', 'NoSeWa', nan], dtype=object)

In [35]:
all_data.Utilities.value_counts()

AllPub    2916
NoSeWa       1
Name: Utilities, dtype: int64

In [36]:
#all_data['Utilities']
#all the data are ALLPub so better to dop this feature (is it a asumption?)


In [37]:
#Drop Feature 
all_data.drop('Utilities', inplace=True, axis=1)

In [38]:
all_data.Functional.unique()

array(['Typ', 'Min1', 'Maj1', 'Min2', 'Mod', 'Maj2', 'Sev', nan],
      dtype=object)

In [39]:
all_data.Functional.isnull().sum()

2

In [40]:
all_data["Functional"] = all_data["Functional"].fillna("Typ")

In [41]:
all_data.Exterior1st.fillna(value='Other', inplace=True)
all_data.Exterior2nd.fillna(value='Other', inplace=True)

In [42]:
all_data.SaleType.isnull().sum()

1

In [43]:
all_data.SaleType.fillna(value='Other', inplace=True)

In [44]:
all_data['Electrical'].unique()

array(['SBrkr', 'FuseF', 'FuseA', 'FuseP', 'Mix', nan], dtype=object)

In [45]:
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
#imput it with mode value
#Mode imputation (or mode substitution) replaces missing values of a categorical variable 
#by the mode of non-missing cases of that variable.

In [46]:
all_data['Electrical'].unique()

array(['SBrkr', 'FuseF', 'FuseA', 'FuseP', 'Mix'], dtype=object)

In [47]:
all_data['KitchenQual'].unique()

array(['Gd', 'TA', 'Ex', 'Fa', nan], dtype=object)

In [48]:
all_data.KitchenQual.isnull().sum()

1

In [49]:

mode_val = all_data.KitchenQual.value_counts().idxmax()   
all_data['KitchenQual'].fillna(value=mode_val, inplace=True)

In [50]:
#check for other s missing values

null=pd.DataFrame(all_data.isnull().sum(),columns=["Null Values"])
null["% Missing Values"]=(all_data.isna().sum()/len(all_data)*100)
null = null[null["% Missing Values"] > 0]
null.style.background_gradient(cmap='viridis',low =0.2,high=0.1)
#will take care of sales price later on

,Null Values,% Missing Values
SalePrice,1459,49.9829


# Finding about Correlations in Data

In [51]:
describeNum = train_data.describe(include =['float64', 'int64', 'float', 'int'])
describeNum.T.style.background_gradient(cmap='viridis',low=0.2,high=0.1)

,count,mean,std,min,25%,50%,75%,max
MSSubClass,1460,56.8973,42.3006,20,20,50,70,190
LotFrontage,1201,70.05,24.2848,21,59,69,80,313
LotArea,1460,10516.8,9981.26,1300,7553.5,9478.5,11601.5,215245
OverallQual,1460,6.09932,1.383,1,5,6,7,10
OverallCond,1460,5.57534,1.1128,1,5,5,6,9
YearBuilt,1460,1971.27,30.2029,1872,1954,1973,2000,2010
YearRemodAdd,1460,1984.87,20.6454,1950,1967,1994,2004,2010
MasVnrArea,1452,103.685,181.066,0,0,0,166,1600
BsmtFinSF1,1460,443.64,456.098,0,0,383.5,712.25,5644
BsmtFinSF2,1460,46.5493,161.319,0,0,0,0,1474


In [52]:
#plt.figure(figsize=(30,20))
#ax = sns.heatmap(data = train_data.corr(),cmap='YlGnBu',annot=True)

#bottom, top = ax.get_ylim()
#ax.set_ylim(bottom + 0.5,top - 0.5)

In [53]:
#sales price has highest correlation with OVerallQual
#find those that highest correlations
print(train_data.corr().nlargest(20,'SalePrice')['SalePrice'].values)


[1.         0.7909816  0.70862448 0.6404092  0.62343144 0.61358055
 0.60585218 0.56066376 0.53372316 0.52289733 0.50710097 0.48636168
 0.47749305 0.46692884 0.38641981 0.3517991  0.32441344 0.3193338
 0.31585623 0.28410768]


In [54]:
print(train_data.corr().nlargest(20,'SalePrice')['SalePrice'])



SalePrice       1.000000
OverallQual     0.790982
GrLivArea       0.708624
GarageCars      0.640409
GarageArea      0.623431
TotalBsmtSF     0.613581
1stFlrSF        0.605852
FullBath        0.560664
TotRmsAbvGrd    0.533723
YearBuilt       0.522897
YearRemodAdd    0.507101
GarageYrBlt     0.486362
MasVnrArea      0.477493
Fireplaces      0.466929
BsmtFinSF1      0.386420
LotFrontage     0.351799
WoodDeckSF      0.324413
2ndFlrSF        0.319334
OpenPorchSF     0.315856
HalfBath        0.284108
Name: SalePrice, dtype: float64


In [55]:
#scatterplot
#sns.set()
#plt.figure(figsize=(30,30))
#cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt', 'YearRemodAdd', 'Fireplaces','BsmtFinSF1', 'WoodDeckSF','OpenPorchSF','2ndFlrSF']
#sns.pairplot(train_data[cols], height = 2.5)
#plt.show()

In [56]:
#my assumption, based on this plot, the salePrice is highly correletad with OverallQual, GrLivearea, 2ndFlrSF 

In [57]:
#sns.set(font_scale=2)
#sns.catplot(y='SalePrice',x='OverallQual',data= all_data.sort_values('SalePrice',ascending=False),kind="boxen",height=6, aspect=3)
#plt.show
#what is overallQual : Rates the overall material and finish of the house

# Label encoding the categorical data


In [58]:
#for handling categorical data, use label encoding or onehat encoding


In [59]:
cats = train_data.describe(include=["O"])
for col in cats:
    print(f'''Value count colunm {col}:''')
    print(train_data[col].value_counts())
    print()

Value count colunm MSZoning:
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64

Value count colunm Street:
Pave    1454
Grvl       6
Name: Street, dtype: int64

Value count colunm Alley:
Grvl    50
Pave    41
Name: Alley, dtype: int64

Value count colunm LotShape:
Reg    925
IR1    484
IR2     41
IR3     10
Name: LotShape, dtype: int64

Value count colunm LandContour:
Lvl    1311
Bnk      63
HLS      50
Low      36
Name: LandContour, dtype: int64

Value count colunm Utilities:
AllPub    1459
NoSeWa       1
Name: Utilities, dtype: int64

Value count colunm LotConfig:
Inside     1052
Corner      263
CulDSac      94
FR2          47
FR3           4
Name: LotConfig, dtype: int64

Value count colunm LandSlope:
Gtl    1382
Mod      65
Sev      13
Name: LandSlope, dtype: int64

Value count colunm Neighborhood:
NAmes      225
CollgCr    150
OldTown    113
Edwards    100
Somerst     86
Gilbert     79
NridgHt     77
Sawyer      74
NWAmes 

In [60]:
#label encoding for ordinal
quality_map = {
    "None": 0, "Po": 1,"Fa": 2, "TA": 3, "Gd": 4, "Ex": 5
}
all_data['GarageQual'].replace(quality_map, inplace=True)
all_data['GarageCond'].replace(quality_map, inplace=True)

In [61]:
quality_map = {
    "None": 0, "Unf": 1, "RFn": 2, "Fin": 3
}

all_data['GarageFinish'].replace(quality_map, inplace=True)

In [62]:
quality_map = {
    "None": 0, "Po": 1,"Fa": 2, "TA": 3, "Gd": 4, "Ex": 5
}

all_data['BsmtCond'].replace(quality_map, inplace=True)
all_data['BsmtQual'].replace(quality_map, inplace=True)

In [63]:
quality_map = {
    "None": 0, "No": 1, "Mn": 2, "Av": 3, "Gd": 4
}

all_data['BsmtExposure'].replace(quality_map, inplace=True)

In [64]:
quality_map = {
    "None": 0, "Unf": 1, "LwQ": 2, "Rec": 3, "BLQ": 4, "ALQ": 5, "GLQ": 6
}

all_data['BsmtFinType1'].replace(quality_map, inplace=True)

In [65]:
quality_map = {
    "None": 0, "Unf": 1, "LwQ": 2, "Rec": 3, "BLQ": 4, "ALQ": 5, "GLQ": 6
}

all_data['BsmtFinType2'].replace(quality_map, inplace=True)

In [66]:
quality_map = {
    "Po": 0,"Fa": 1, "TA": 2, "Gd": 3, "Ex": 4
}
all_data['KitchenQual'].replace(quality_map, inplace=True)

In [67]:
all_data['Street'].unique()

array(['Pave', 'Grvl'], dtype=object)

In [68]:
all_data['Street'].replace({'Grvl': 0, 'Pave': 1}, inplace=True)

In [69]:
#all_data['CentralAir'].unique()
all_data['CentralAir'].replace({'Y': 1, 'N': 0}, inplace=True)

In [70]:
''' HeatingQC | ExterQual | ExterCond'''
quality_map = {
    "Po": 0,"Fa": 1, "TA": 2, "Gd": 3, "Ex": 4
}
all_data.HeatingQC.replace(quality_map, inplace=True)
all_data.ExterQual.replace(quality_map, inplace=True)
all_data.ExterCond.replace(quality_map, inplace=True)

''' LandSlope '''
quality_map = { 'Sev': 0, 'Mod': 1, 'Gtl': 2 }
all_data.LandSlope.replace(quality_map, inplace=True)

''' LotShape '''
quality_map = { 'IR3': 0, 'IR2': 1, 'IR1': 2, 'Reg': 3 }
all_data.LotShape.replace(quality_map, inplace=True)

In [71]:
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")

In [72]:
class_map = {
    20:'Class1', 30:'Class2', 40:'Class3', 45:'Class4', 50:'Class5', 60:'Class6', 
    70:'Class7', 75:'Class8', 80:'Class9', 85:'Class10', 90:'Class11', 120:'Class12',
    150:'Class13', 160:'Class14', 180:'Class15', 190:'Class16'
}
all_data.MSSubClass.replace(class_map, inplace=True)

In [73]:
cat_cols = all_data.select_dtypes("object").columns

In [99]:
cat_cols

Index(['Alley', 'BldgType', 'Condition1', 'Condition2', 'Electrical',
       'Exterior1st', 'Exterior2nd', 'Fence', 'FireplaceQu', 'Foundation',
       'Functional', 'GarageType', 'Heating', 'HouseStyle', 'LandContour',
       'LotConfig', 'MSSubClass', 'MSZoning', 'MasVnrType', 'MiscFeature',
       'Neighborhood', 'PavedDrive', 'RoofMatl', 'RoofStyle', 'SaleCondition',
       'SaleType'],
      dtype='object')

In [75]:
## One-Hot Encoding all the nominal  categorical variables but dropping one of the features among them.
drop_categ = []
for i in cat_cols:
    drop_categ += [ i+'_'+str(all_data[i].unique()[-1]) ]

## Create dummy variables (One-Hot Encoding)
all_data = pd.get_dummies(all_data, columns=cat_cols) 

## Drop the last column generated from each categorical feature
all_data.drop(drop_categ, axis=1, inplace=True)

#dummies for nominal

In [76]:
all_data['TotalSF'] =  all_data['GrLivArea'] + all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

In [77]:
all_data['AgeofHouse'] = all_data.YrSold - all_data.YearBuilt

In [78]:
#all_data['AgeofHouse'].unique()

In [79]:
# replace this value with 0 to avoid negative values.
all_data.AgeofHouse = all_data.AgeofHouse.map(lambda x: 0 if x < 0 else x)

In [80]:
#all_data.columns

In [81]:
#check Nan values in sales price data
#train_data.SalePrice.isnull().sum() #Nan value is in Test data not train data

# check for duplication and negative vaules

In [82]:
#check
duplicate = all_data[all_data.duplicated()] 
  
print("Duplicate Rows :")

Duplicate Rows :


In [83]:
c = all_data.select_dtypes(exclude = ["object"]).columns

#for a in range(len(c)):
    #print("Is there any negative value in '{}' column  : {} ".format(c[a],(all_data[c[a]]<0).any()))
#there is nothing

# Feature importance

In [84]:
X_train =  all_data[:-ntest].drop(['SalePrice'], axis=1)
y_train =  all_data[:-ntest]['SalePrice']
X_test  =  all_data[-ntest:].drop(['SalePrice'], axis=1)


In [85]:

# linear regression feature importance
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
xgb = XGBRegressor()
# define the model
model = LinearRegression()
# fit the model
model.fit(X_train, y_train)
# get importance
importance = model.coef_
df_imp = pd.DataFrame(importance , columns = ['Importance'], index=X_train.columns)
df_imp = df_imp.sort_values(['Importance'], ascending = False)
#df_imp[ df_imp.iloc[:,0:1]> 1000 ]


In [95]:
cols = df_imp[df_imp['Importance'] >= 5]
#i made the assumption that features that has importance less than 1000 are not important
#so better to drop them
ind = cols.index
cols.head()

,Importance
BsmtFinSF1,1.368668e+13
BsmtFinSF2,1.368668e+13
BsmtUnfSF,1.368668e+13
GrLivArea,4.176546e+12
YearBuilt,6.496810e+11


In [88]:
X_train = X_train[ind]
X_test = X_test[ind]

# SKEWNESS 

In [89]:
## Get all the numeric features in out dataset
numeric_features = all_data.skew().index

## Getting all the skewed features (skew > 0.5 or skew < -0.5)
skewed_features = all_data[numeric_features].skew()[np.abs(all_data[numeric_features].skew()) > 0.5].index

## Performing log(1+x) transformation
all_data[skewed_features] = np.log1p(all_data[skewed_features])

# Performing machine learning Models on the data

In [90]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

#import xgboost as xgb
#import lightgbm as lgb

In [91]:
#scaling- standardizing the data
#get numeric features
numeric_features = X_train.skew().index
#scaling the numeric features
#numeric_features = numeric_features.drop('SalePrice')

# data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler
scaler = RobustScaler()
#
X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_test[numeric_features]  = scaler.transform(X_test[numeric_features])

In [92]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [93]:
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)
y_pred= linear_reg.predict(X_test)
print("Applied Model : ",LinearRegression)
print("Accuracy on Training set   : ",linear_reg.score(X_train,y_train))
print("Accuracy on Testing set  : ",linear_reg.score(X_test,y_test))
print("__________________________________________")
print("\t\tError Table")
print('Mean Absolute Error      : ', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error       : ', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error  : ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R Squared Error          : ', metrics.r2_score(y_test, y_pred))

Applied Model :  <class 'sklearn.linear_model.base.LinearRegression'>
Accuracy on Training set   :  0.9242400184876339
Accuracy on Testing set  :  0.7365073196153991
__________________________________________
		Error Table
Mean Absolute Error      :  0.10306967309773954
Mean Squared Error       :  0.04917062957254401
Root Mean Squared Error  :  0.22174451418816207
R Squared Error          :  0.7365073196153991


In [94]:
RandomForest = RandomForestRegressor()
RandomForest.fit(X_train, y_train)
y_pred= RandomForest.predict(X_test)
print("Applied Model : ",RandomForest)
print("Accuracy on Traing set   : ",RandomForest.score(X_train,y_train))
print("Accuracy on Testing set  : ",RandomForest.score(X_test,y_test))
print("__________________________________________")
print("\t\tError Table")
print('Mean Absolute Error      : ', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error       : ', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error  : ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R Squared Error          : ', metrics.r2_score(y_test, y_pred))

Applied Model :  RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
Accuracy on Traing set   :  0.9696463127679957
Accuracy on Testing set  :  0.8560178415278712
__________________________________________
		Error Table
Mean Absolute Error      :  0.11505599188410627
Mean Squared Error       :  0.026868652931666494
Root Mean Squared Error  :  0.16391660358751486
R Squared Error          :  0.8560178415278713


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
